# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 62 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.02712


extracting tarball to tmp_2209.02712...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.02722


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.02712/SMS.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2209.03215...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.03278


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.03215/NGC346.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.03278...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.02722-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.02722) | **The Poor Old Heart of the Milky Way**  |
|| <mark>Hans-Walter Rix</mark>, et al. -- incl., <mark>René Andrae</mark>, <mark>Morgan Fouesneau</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *21 pages, 9 figures, submitted to ApJ*|
|**Abstract**| Massive disk galaxies like our Milky Way should host an ancient, metal-poor, and centrally concentrated stellar population. This population reflects the star formation and enrichment in the few most massive progenitor components that coalesced at high redshift to form the proto-Galaxy. While metal-poor stars are known to reside in the inner few kiloparsecs of our Galaxy, current data do not yet provide a comprehensive picture of such a metal-poor "heart" of the Milky Way. We use information from Gaia DR3, especially the XP spectra, to construct a sample of 2 million bright (BP $<15.5$ mag) giant stars within $30^\circ$ of the Galactic Center with robust [M/H] estimates, $\delta$ [M/H] $\lesssim 0.1$. For most sample members we can calculate orbits based on Gaia RVS velocities and astrometry. This sample reveals an extensive, ancient, and metal-poor population that includes $\sim 18,000$ stars with $-2.7<$ [M/H] $<-1.5$, representing a stellar mass of $\gtrsim 5\times 10^7$ M$_\odot$. The spatial distribution of these [M/H] $<-1.5$ stars has a Gaussian extent of only $\sigma_{\mathrm{R_{GC}}} \sim 2.7$ kpc around the Galactic center, with most of these orbits being confined to the inner Galaxy. At high orbital eccentricities, there is clear evidence for accreted halo stars in their pericentral orbit phase. Stars with [M/H] $< -2$ show no net rotation, whereas those with [M/H] $\sim -1$ are rotation dominated. Most of the tightly bound stars show $[\alpha/\text{Fe}]$-enhancement and [Al/Fe]-[Mn/Fe] abundance patterns expected for an origin in the more massive portions of the proto-Galaxy. These central, metal-poor stars most likely predate the oldest part of the disk ($\tau_{\text{age}}\approx 12.5$ Gyrs), which implies that they formed at $z\gtrsim 5$, forging the proto-Milky Way. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.02725-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.02725) | **A Generative Model for Quasar Spectra**  |
|| Anna-Christina Eilers, et al. -- incl., <mark>Frederick B. Davies</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *accepted for publication in ApJ*|
|**Abstract**| We build a multi-output generative model for quasar spectra and the properties of their black hole engines, based on a Gaussian process latent-variable model. This model treats every quasar as a vector of latent properties such that the spectrum and all physical properties of the quasar are associated with non-linear functions of those latent parameters; the Gaussian process kernel functions define priors on the function space. Our generative model is trained with a justifiable likelihood function that allows us to treat heteroscedastic noise and missing data correctly, which is crucial for all astrophysical applications. It can predict simultaneously unobserved spectral regions, as well as the physical properties of quasars in held-out test data. We apply the model to rest-frame ultraviolet and optical quasar spectra for which precise black hole masses (based on reverberation mapping measurements) are available. Unlike reverberation-mapping studies, which require multi-epoch data, our model predicts black hole masses from single-epoch spectra, even with limited spectral coverage. We demonstrate the capabilities of the model by predicting black hole masses and unobserved spectral regions. We find that we predict black hole masses at close to the best possible accuracy. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.02872-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.02872) | **PHANGS: Constraining Star Formation Timescales Using the Spatial  Correlations of Star Clusters and Giant Molecular Clouds**  |
|| Jordan A. Turner, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>J. M. Diederik Kruijssen</mark>, <mark>Thomas G. Williams</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *15 pages, 11 figures, 4 tables. Accepted to MNRAS Sept 6 2022*|
|**Abstract**| In the hierarchical view of star formation, giant molecular gas clouds (GMCs) undergo fragmentation to form small-scale structures made up of stars and star clusters. Here we study the connection between young star clusters and cold gas across a range of extragalactic environments by combining the high resolution (1") PHANGS-ALMA catalogue of GMCs with the star cluster catalogues from PHANGS-HST. The star clusters are spatially matched with the GMCs across a sample of 11 nearby star-forming galaxies with a range of galactic environments (centres, bars, spiral arms, etc.). We find that after 4-6 Myr the star clusters are no longer associated with any gas clouds. Additionally, we measure the autocorrelation of the star clusters and GMCs as well as their cross-correlation to quantify the fractal nature of hierarchical star formation. Young ($\leq$ 10 Myr) star clusters are more strongly autocorrelated on kpc and smaller spatial scales than the >10 Myr stellar populations, indicating that the hierarchical structure dissolves over time. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03088-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03088) | **Probing the Mpc-scale environment of hyperluminous infrared galaxies at  2<z<4**  |
|| F. Gao, et al. -- incl., <mark>A. F. Ramos Padilla</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *19 pages, 10 figures, accepted by A&A*|
|**Abstract**| Protoclusters are important for studying how halo mass and stellar mass assemble in the early universe. Finding signposts of such over-dense regions is a popular method to identify protocluster candidates. Hyperluminous infrared galaxies (HLIRGs), are expected to reside in overdense regions with massive halos. We study the Mpc-scale environment of the largest HLIRG sample to date and investigate whether they predominantly live in overdense regions. We first explore the surface density of Herschel 250 $\mu$m sources around HLIRGs and compare with that around random positions. Then, we compare the spatial distribution of neighbours around HLIRGs with that around randomly selected galaxies using a deep IRAC-selected catalogue with good-quality photometric redshifts. We also use a redshift-matched quasar sample and submillimeter galaxy (SMG) sample to validate our method, as previous clustering studies have measured the host halo masses of these populations. Finally, we adopt a Friends of Friends (FOF) algorithm to seek (proto)clusters that host HLIRGs. We find that HLIRGs tend to have more bright star-forming neighbours (with 250 $\mu$m flux density >10 mJy) within 100$\arcsec$ projected radius than a random galaxy at a 3.7$\sigma$ significance. In our 3D analysis, we find relatively weak excess of IRAC-selected sources within 3 Mpc around HLIRGs compared with random galaxy neighbours, mainly influenced by photometric redshift uncertainty and survey depth. We find a more significant difference (at a 4.7$\sigma$ significance) in the number of Low Frequency Array (LOFAR)-detected neighbours in the deepest EN1 field. HLIRGs at 3 < z < 4 show stronger excess compared to HLIRGs at 2 < z < 3, consistent with cosmic downsizing. Finally, we select and present a list of 30 most promising protocluster candidates for future follow-up observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03128-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03128) | **The EBLM project -- IX. Five fully convective M-dwarfs, precisely  measured with CHEOPS and TESS light curves**  |
|| D. Sebastian, et al. -- incl., <mark>G. Olofsson</mark>, <mark>S. Hoyer</mark>, <mark>N. Heidari</mark>, <mark>M. B. Davies</mark>, <mark>K. Heng</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *19 pages, 12 figures, accepted for publication in MNRAS*|
|**Abstract**| Eclipsing binaries are important benchmark objects to test and calibrate stellar structure and evolution models. This is especially true for binaries with a fully convective M-dwarf component for which direct measurements of these stars' masses and radii are difficult using other techniques. Within the potential of M-dwarfs to be exoplanet host stars, the accuracy of theoretical predictions of their radius and effective temperature as a function of their mass is an active topic of discussion. Not only the parameters of transiting exoplanets but also the success of future atmospheric characterisation rely on accurate theoretical predictions. We present the analysis of five eclipsing binaries with low-mass stellar companions out of a sub-sample of 23, for which we obtained ultra high-precision light curves using the CHEOPS satellite. The observation of their primary and secondary eclipses are combined with spectroscopic measurements to precisely model the primary parameters and derive the M-dwarfs mass, radius, surface gravity, and effective temperature estimates using the PYCHEOPS data analysis software. Combining these results to the same set of parameters derived from TESS light curves, we find very good agreement (better than 1\% for radius and better than 0.2% for surface gravity). We also analyse the importance of precise orbits from radial velocity measurements and find them to be crucial to derive M-dwarf radii in a regime below 5% accuracy. These results add five valuable data points to the mass-radius diagram of fully-convective M-dwarfs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03161-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03161) | **Status of the PSF Reconstruction Work Package for MICADO ELT**  |
|| Andrea Grazian, et al. -- incl., <mark>Richard Davies</mark>, <mark>Tapio Helin</mark>, <mark>Steven Williams</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *to appear in the Proceedings of the SPIE conference Astronomical Telescopes+Instrumentation 2022 Montreal, Quebec, Canada; 6 pages, 1 figure, 1 table*|
|**Abstract**| MICADO is a workhorse instrument for the ESO ELT, allowing first light capability for diffraction limited imaging and long-slit spectroscopy at near-infrared wavelengths. The PSF Reconstruction (PSF-R) Team of MICADO is currently implementing, for the first time within all ESO telescopes, a software service devoted to the blind reconstruction of the PSF. This tool will work independently of the science data, using adaptive optics telemetry data, both for Single Conjugate (SCAO) and Multi-Conjugate Adaptive Optics (MCAO) allowed by the MORFEO module. The PSF-R service will support the state-of-the-art post-processing scientific analysis of the MICADO imaging and spectroscopic data. We provide here an update of the status of the PSF-R service tool of MICADO, after successfully fulfilling the Final Design Review phase, and discuss recent results obtained on simulated and real data gathered on instruments similar to MICADO. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03215-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03215) | **The internal proper motion kinematics of NGC346: past formation and  future evolution**  |
|| E. Sabbi, et al. -- incl., <mark>L.J. Smith</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *17 pages, 14 figures, Accpeted for publication on ApJ*|
|**Abstract**| We investigate the internal kinematics of the young star-forming region NGC 346 in the Small Magellanic Cloud. We used two epochs of deep F555W and F814W Hubble Space Telescope ACS observations with an 11-year baseline to determine proper motions, and study the kinematics of different populations, as identified by their color-magnitude diagram and spatial distribution characteristics. The proper motion field of the young stars shows a complex structure with spatially coherent patterns. NGC 346 upper-main sequence and pre-main sequence stars follow very similar motion patterns, with the outer parts of the cluster being characterized both by outflows and inflows. The proper motion field in the inner ~10 pc shows a combination of rotation and inflow, indicative of inspiraling motion. The rotation velocity in this regions peaks at ~3 km/s, whereas the inflow velocity peaks at ~1 km/s. Sub-clusters and massive young stellar objects in NGC 346 are found at the interface of significant changes in the coherence of the proper motion field. This suggests that turbulence is the main star formation driver in this region. The similar kinematics observed in the metal-poor NGC 346 and the Milky Way star-forming regions suggest that the differences in the cooling conditions due to the different amounts of metallicity and dust density between the SMC and our Galaxy are too small to alter significantly the process of star clusters assembly and growth. The main characteristics of our findings are consistent with various proposed star cluster formation models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03278-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03278) | **BRUTE, PSF Reconstruction for the SOUL pyramid-based Single Conjugate  Adaptive Optics facility of the LBT**  |
|| Carmelo Arcidiacono, et al. -- incl., <mark>Enrico Pinna</mark> |
|*Appeared on*| *2022-09-08*|
|*Comments*| *11 pages, 7 figures, Proceeding of the SPIE Conference 12185, Adaptive Optics Systems VIII, 1218540 (29 August 2022)*|
|**Abstract**| The astronomical applications greatly benefit from the knowledge of the instrument PSF. We describe the PSF Reconstruction algorithm developed for the LBT LUCI instrument assisted by the SOUL SCAO module. The reconstruction procedure considers only synchronous wavefront sensor telemetry data and a few asynchronous calibrations. We do not compute the Optical Transfer Function and corresponding filters. We compute instead a temporal series of wavefront maps and for each of these the corresponding instantaneous PSF. We tested the algorithm both in laboratory arrangement and in the nighttime for different SOUL configurations, adapting it to the guide star magnitudes and seeing conditions. We nick-named it "BRUTE", Blind Reconstruction Using TElemetry, also recalling the one-to-one approach, one slope-to one instantaneous PSF the algorithm applies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.02722.md
    + _build/html/tmp_2209.02722/./MH_validation.png
    + _build/html/tmp_2209.02722/./dust_validation.png
    + _build/html/tmp_2209.02722/./LAMOST_MH_validation.png
    + _build/html/tmp_2209.02722/./GALAH_MH_validation.png
    + _build/html/tmp_2209.02722/./GSPSpec_MH_validation.png
    + _build/html/tmp_2209.02722/./Teff_validation.png
    + _build/html/tmp_2209.02722/./BP_validation.png
exported in  _build/html/2209.02725.md
    + _build/html/tmp_2209.02725/./1to1_0_f_ps_s42_HST_1220_5000_2A_band1fixed_noSNRcut_beta10_new_2kernels_cross_N30_D1889_L2_Q16_SNR_overlap.png
    + _build/html/tmp_2209.02725/./spectra_ps_s42_HST_1220_5000_2A_band1fixed_noSNRcut_beta10_new_2kernels_cross_N30_D1889_L3_Q16_qq4_qq4_new.png
    + _build/html/tmp_2209.02725/./sample.png
exported in  _build/html/2209.02872.md
    + _build/html/tmp_2209.02872/./figures/tpcf_grid_jan15.ps
    + _build/html/tmp_2209.02872/./figures/assoc_1566.ps
    + _build/html/tmp_2209.02

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\insitu}{{\it in situ}}$
$\newcommand{\MH}{\ensuremath{\mathrm{[M/H]}}}$
$\newcommand{\Teff}{\ensuremath{\mathrm{T}_{\rm eff}}}$
$\newcommand{\aFe}{\ensuremath{\mathrm{[\alpha/M]}}}$
$\newcommand{\GBP}{\ensuremath{G_\mathrm{BP}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\insitu$}{{\it in situ}}$
$\newcommand{$\MH$}{$\ensuremath${\mathrm{[M/H]}}}$
$\newcommand{$\Teff$}{$\ensuremath${\mathrm{T}_{\rm eff}}}$
$\newcommand{$\aFe$}{$\ensuremath${\mathrm{[\alpha/M]}}}$
$\newcommand{$\GBP$}{$\ensuremath${G_\mathrm{BP}}}$</div>



<div id="title">

# The Poor Old Heart of the Milky Way

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.02722-b31b1b.svg)](https://arxiv.org/abs/2209.02722)<mark>Appeared on: 2022-09-08</mark> - _21 pages, 9 figures, submitted to ApJ_

</div>
<div id="authors">

<mark>Hans-Walter Rix</mark>, et al. -- incl., <mark>Ren\'e Andrae</mark>, <mark>Morgan Fouesneau</mark>

</div>
<div id="abstract">

**Abstract:** Massive disk galaxies like our Milky Way should host an ancient, metal-poor, and centrally concentrated stellar population. This population reflects the star formation and enrichment in the few most massive progenitor components that coalesced at high redshift to form the\emph{proto-Galaxy}. While metal-poor stars are known to reside in the inner few kiloparsecs of our Galaxy, current data do not yet provide a comprehensive picture of such a metal-poor "heart" of the Milky Way. We use information from Gaia DR3, especially the XP spectra, to construct a sample of 2 million bright ($\GBP<15.5$mag) giant stars within$30^\circ$of the Galactic Center with robust$\MH$estimates,$\delta$\MH$\lesssim 0.1$. For most sample members we can calculate orbits based on Gaia RVS velocities and astrometry. This sample reveals an extensive, ancient, and metal-poor population that includes$\sim 18,000$stars with$-2.7<$\MH$<-1.5$, representing a stellar mass of$\gtrsim 5\times 10^7$M$_\odot$. The spatial distribution of these$\MH<-1.5$stars has a Gaussian extent of only$\sigma_{\mathrm{R_{GC}}} \sim 2.7$kpc around the Galactic center, with most of these orbits being confined to the inner Galaxy.  At high orbital eccentricities, there is clear evidence for\emph{accreted}halo stars in their pericentral orbit phase.Stars with$\MH< -2$show no net rotation, whereas those with$\MH\sim -1$are rotation dominated.Most of the tightly bound stars show$[\alpha/\text{Fe}]$-enhancement and[Al/Fe]--[Mn/Fe]abundance patterns expected for an origin in the more massive portions of the proto-Galaxy.These central, metal-poor stars most likely predate the oldest part of the disk ($\tau_{\text{age}}\approx 12.5$Gyrs), which implies that they formed at$z\gtrsim 5$, forging the proto-Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2209.02722/./MH_validation.png" alt="Fig1.1" width="50%"/><img src="tmp_2209.02722/./dust_validation.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Top: Validation of our $\MH$ estimates from \textit{Gaia} XP data as function of actual [M/H] from APOGEE.
Bottom: (In-)sensitivity of $\MH$ estimates to extinction, $A_K$. The Y-axis shows the difference between $\MH$ estimates based on XP spectra and APOGEE as a function of APOGEE's $A_K$. There is no evidence for any systematic trend of $\Delta$\MH$$ with $A_K$ to an extinction level that corresponds to $A_V \approx 3$. (*fig:MH_validation*)

</div>
<div id="div_fig2">

<img src="tmp_2209.02722/./LAMOST_MH_validation.png" alt="Fig3.1" width="33%"/><img src="tmp_2209.02722/./GALAH_MH_validation.png" alt="Fig3.2" width="33%"/><img src="tmp_2209.02722/./GSPSpec_MH_validation.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Cross-validation of our $\MH$$_{\text{XP}}$  estimates against three external spectroscopic data sets, LAMOST \citep[here][]{Xiang2019} , GALAH \citep{GALAH_DR3} and {\tt GSP-Spec}\citep{Recio-Blanco-RVS-abundances}. This cross-validation supports the precision and robustness of our $\MH$$_{\text{XP}}$ estimates. (*fig:MH_cross_validation*)

</div>
<div id="div_fig3">

<img src="tmp_2209.02722/./Teff_validation.png" alt="Fig2.1" width="50%"/><img src="tmp_2209.02722/./BP_validation.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Top: Validation of our T$_{\text{eff}}$(XP) estimates against the T$_{\text{eff}}$ from SDSS DR17 APOGEE. The XP predictions match the APOGEE values within a median $\Delta$T$_{eff}=32$ K, and with a mean difference of only 3 K. Bottom: Validation of our $\MH$ estimates as a function of G$_{\text{BP}}$ magnitude, with $\Delta$\MH$ \equiv $\MH$_{\text{XP}} - $\MH$_{\text{APOGEE}}$, showing that the quality of the estimate depends somewhat on G$_{\text{BP}}$, as expected, but is overall unbiased. (*fig:BP_validation*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{\lya}{Ly\alpha }$
$\newcommand{\lyb}{Ly\beta }$
$\newcommand{\lyc}{Ly\gamma }$
$\newcommand{\taueff}{\tau_{\rm eff} }$
$\newcommand{\cmpch}{cMpc h^{-1} }$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\oi}{\ion{O}{1}}$
$\newcommand{\heii}{\ion{He}{2}}$
$\newcommand{\cii}{[\ion{C}{2}]}$
$\newcommand{\civ}{\ion{C}{4}}$
$\newcommand{\mgii}{\ion{Mg}{2}}$
$\newcommand{\siiv}{\ion{Si}{4}}$
$\newcommand{\siii}{\ion{Si}{2}}$
$\newcommand{\oiii}{[\ion{O}{3}]}$
$\newcommand{\co}{\rm CO (6-5)}$
$\newcommand{\het}{heteroscedasticity}$
$\newcommand{\ha}{H\alpha }$
$\newcommand{\hb}{H\beta }$
$\newcommand{\msr}{M_\bullet-\sigma_\ast~relation}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{$\lya$}{Ly\alpha }$
$\newcommand{$\lyb$}{Ly\beta }$
$\newcommand{$\lyc$}{Ly\gamma }$
$\newcommand{$\taueff$}{\tau_{\rm eff} }$
$\newcommand{$\cmpch$}{cMpc h^{-1} }$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand{$\oi$}{$\ion${O}{1}}$
$\newcommand{$\heii$}{$\ion${He}{2}}$
$\newcommand{$\cii$}{[$\ion${C}{2}]}$
$\newcommand{$\civ$}{$\ion${C}{4}}$
$\newcommand{$\mgii$}{$\ion${Mg}{2}}$
$\newcommand{$\siiv$}{$\ion${Si}{4}}$
$\newcommand{$\siii$}{$\ion${Si}{2}}$
$\newcommand{$\oi$ii}{[$\ion${O}{3}]}$
$\newcommand{$\co$}{\rm CO (6-5)}$
$\newcommand{$\het$}{heteroscedasticity}$
$\newcommand{$\ha$}{H\alpha }$
$\newcommand{$\hb$}{H\beta }$
$\newcommand{$\msr$}{M_\bullet-\sigma_\ast~relation}$</div>



<div id="title">

#  A Generative Model for Quasar Spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.02725-b31b1b.svg)](https://arxiv.org/abs/2209.02725)<mark>Appeared on: 2022-09-08</mark> - _accepted for publication in ApJ_

</div>
<div id="authors">

Anna--Christina~Eilers, et al. -- incl., <mark>Frederick~B.~Davies</mark>

</div>
<div id="abstract">

**Abstract:** \noindentWe build a multi-output generative model for quasar spectra and the properties of their black hole engines, based on a Gaussian process latent-variable model.This model treats every quasar as a vector of latent properties such that the spectrum and all physical properties of the quasar are associated with non-linear functions of those latent parameters; the Gaussian process kernel functions define priors on the function space.Our generative model is trained with a justifiable likelihood function that allows us to treat heteroscedastic noise and missing data correctly, which is crucial for all astrophysical applications.It can predict simultaneously unobserved spectral regions, as well as the physical properties of quasars in held-out test data.We apply the model to rest-frame ultraviolet and optical quasar spectra for which precise black hole masses (based on reverberation mapping measurements) are available.Unlike reverberation-mapping studies, which require multi-epoch data, our model predicts black hole masses from single-epoch spectra, even with limited spectral coverage.We demonstrate the capabilities of the model by predicting black hole masses and unobserved spectral regions. We find that we predict black hole masses at close to the best possible accuracy.

</div>

<div id="div_fig1">

<img src="tmp_2209.02725/./1to1_0_f_ps_s42_HST_1220_5000_2A_band1fixed_noSNRcut_beta10_new_2kernels_cross_N30_D1889_L2_Q16_SNR_overlap.png" alt="Fig2" width="100%"/>

**Figure 2. -** Cross-validation of the GPLVM with $Q=16$ latent dimensions. Predicted compared to measured black hole masses for all quasars in our data set, colored by the SNR of the single-epoch quasar spectrum. Note that we use only two labels for the quasars here, i.e. $L=2$, namely black hole mass and bolometric luminosity.  (*fig:1to1*)

</div>
<div id="div_fig2">

<img src="tmp_2209.02725/./spectra_ps_s42_HST_1220_5000_2A_band1fixed_noSNRcut_beta10_new_2kernels_cross_N30_D1889_L3_Q16_qq4_qq4_new.png" alt="Fig5" width="100%"/>

**Figure 5. -** Spectral dependencies on various quasar properties, i.e. black hole mass (\textit{top}), bolometric luminosity (\textit{middle}) and redshift (\textit{bottom}). The inset panel shows the expected spectral changes due to the Baldwin effect.  (*fig:dependencies*)

</div>
<div id="div_fig3">

<img src="tmp_2209.02725/./sample.png" alt="Fig1" width="100%"/>

**Figure 1. -** Black hole masses and bolometric luminosities colored by redshift of 31 quasars in our data sample, which have reliable black hole mass measurements based on the RM technique.  (*fig:parent_sample*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\HII}{H\small{II}\normalsize }$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\HII$}{H\small{II}\normalsize }$
$\newcommand$
$\newcommand$</div>



<div id="title">

# PHANGS: Constraining Star Formation Timescales Using the Spatial Correlations of Star Clusters and Giant Molecular Clouds

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.02872-b31b1b.svg)](https://arxiv.org/abs/2209.02872)<mark>Appeared on: 2022-09-08</mark> - _15 pages, 11 figures, 4 tables. Accepted to MNRAS Sept 6 2022_

</div>
<div id="authors">

Jordan A. Turner, et al. -- incl., <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Annie Hughes</mark></mark>, <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>J. M. Diederik Kruijssen</mark></mark>, <mark><mark>Thomas G. Williams</mark></mark>

</div>
<div id="abstract">

**Abstract:** In the hierarchical view of star formation, giant molecular gas clouds (GMCs) undergo fragmentation to form small-scale structures made up of stars and star clusters. Here we study the connection between young star clusters and cold gas across a range of extragalactic environments by combining the high resolution (1$\arcsec$) PHANGS--ALMA catalogue of GMCs with the star cluster catalogues from PHANGS--HST. The star clusters are spatially matched with the GMCs across a sample of$11$nearby star-forming galaxies with a range of galactic environments (centres, bars, spiral arms, etc.).  We find that after$4{-}6$Myr the star clusters are no longer associated with any gas clouds. Additionally, we measure the autocorrelation of the star clusters and GMCs as well as their cross-correlation to quantify the fractal nature of hierarchical star formation. Young (${\leq}10$Myr) star clusters are more strongly autocorrelated on kpc and smaller spatial scales than the$>10$Myr stellar populations, indicating that the hierarchical structure dissolves over time.

</div>

<div id="div_fig1">

<img src="tmp_2209.02872/./figures/tpcf_grid_jan15.ps" alt="Fig10" width="100%"/>

**Figure 10. -** Two-point (auto) correlation functions for all star clusters (black), clusters ${\leq}10$ Myr (blue), clusters ${>}10$ Myr (red), and GMCs (orange). The line at $1 + \omega(\theta) = 1$ marks where a uniform, random distribution. Twice the median radius of the GMCs in each galaxy is shown as the vertical grey line. Uncertainties are bootstrap estimates. The thick grey line marks the distance range over which the power-laws are fit; the best-fitting power-law slopes are given in Table \ref{tab:tpcf_plaw}. Correlation functions which exhibit anti-correlation ($1 + \omega(\theta) < 1$) are most likely artifacts of the random sampling i.e., there are more random points than data points for that angular bin. In which case, correlation at these bins can be considered consistent with a random distribution. (*fig:tpcf_grid*)

</div>
<div id="div_fig2">

<img src="tmp_2209.02872/./figures/assoc_1566.ps" alt="Fig2" width="100%"/>

**Figure 2. -** Histograms of the star cluster ages sorted according to association with GMCs in NGC 1566: all star clusters (grey), within one GMC radius (blue), between 1 and 2 GMC radii (green), between 2 and 3 GMC radii (orange), and beyond 3 GMC radii which is considered unassociated (fuchsia). Vertical dashed lines mark the median cluster age for each distribution. A $10$ km s$^{-1}$ velocity cutoff is applied to minimize counting clusters which have random alignment with the GMCs. (*fig:assoc_hist*)

</div>
<div id="div_fig3">

<img src="tmp_2209.02872/./figures/1nn_grid.ps" alt="Fig7" width="100%"/>

**Figure 7. -** Histograms of the separations between star clusters and the centre of the nearest neighbour GMC for each of the 11 galaxies and an ensemble of all the galaxies. The distribution for the full sample of star clusters is shown in grey, for clusters ${\leq}10$ Myr is blue, and for clusters ${>}10$ Myr is red. Median separations for each distribution are marked with dashed lines.  Theoretical estimates for the nearest neighbor separations for NGC 0628, NGC 3351, NGC 3627, NGC 4535, and the ensemble are provided by green dotted lines (see text).  The histograms are normalized by the total number of counts and the bin width for each histogram. (*fig:1nn_grid*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

28  publications in the last 7 days.
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers